In [1]:
from py_api import Experiment
import numpy as np

In [2]:
my_exp = Experiment()

In [3]:
my_exp

Experiment class.
The following AO cards have been added already:
{}

The following DO cards have been added already:
{}

In [4]:
fast_ao_card = my_exp.add_card(
    card_type='AO',
    max_name='PXI1Slot3',
    samp_rate=1e6,
    trig_mode=True,
    trig_line='PXI_Trig0',
    ref_clk_mode=True,
    ref_clk_line='PXI_Trig7',
    ref_clk_rate=10e6
)

In [5]:
my_exp['PXI1Slot3']

<class 'py_api.Experiment.AOCard'>  card.
The following output channels have been added:
{}

In [6]:
slow_ao_card = my_exp.add_card(
    card_type='AO',
    max_name='PXI1Slot4',
    samp_rate=250e3,
    trig_mode=False,
    trig_line='PXI_Trig0',
    ref_clk_mode=False,
    ref_clk_line='PXI_Trig7',
    ref_clk_rate=10e6
)

In [7]:
do_card = my_exp.add_card(
    card_type='DO',
    max_name='PXI1Slot6',
    samp_rate=10e6,
    samp_clk_src='PXI_Trig7',
    trig_mode=False,
    trig_line='PXI_Trig0'
)

In [ ]:
# Add channels

In [8]:
bfield_grad = slow_ao_card.add_chnl(
    chan_idx=0
)

In [9]:
slow_ao_card[0]

In [10]:
mot_aom_pwr = fast_ao_card.add_chnl(
    chan_idx=0
)

In [11]:
mot_shutter = do_card.add_chnl(
    port_idx=0,
    line_idx=0
)

In [12]:
do_card['port0/line0']

In [14]:
my_exp.clear_edit_cache()

In [15]:
tau_num = 100
tau_arr = np.linspace(0.5e-3, 10e-3, tau_num)

mot_pwr_num = 10
mot_pwr_arr = np.linspace(0, 1, mot_pwr_num)

buf_time = 100e-6

# Experiment logic script
t = 0

for mot_pwr in mot_pwr_arr:
    for tau in tau_arr:
        
        t += buf_time  # Safety buffer
        
        mot_aom_pwr.sine(
            t=t, dur=tau,
            amp=mot_pwr, freq=100e3,
        )
        
        mot_shutter.go_high(t=t)
        mot_shutter.go_low(t=t+tau)
        
        t += (tau + buf_time)

bfield_grad.sine(
    t=0, dur=t + 1e-3,
    amp=0.5, dc_offs=0.5,
    freq=10e3
)

In [24]:
my_exp.compile()

5.451

In [25]:
my_exp.stream_exp()

In [18]:
my_exp.reset_all()